In [0]:
import os
from scipy.io import loadmat

# Get current notebook directory
notebook_dir = os.getcwd()
file_path = os.path.join(notebook_dir, "test.mat")

input = loadmat(file_path)
a = input["a"].flatten()
x = input["x"].flatten()

![assignement4_a.png](./assignement4_a.png "assignement4_a.png")


#### 1)

In linear predictive coding we try to decompose a signal \\(x[n]\\) into an all pole filter(Resonant) \\(h[n]\\) and the excitation \\(e[n]\\).

$$X(z) = H(z)E(z)$$$$= \frac{1}{1 - \sum_{k=1}^{p} a_k z^{-k}} E(z)$$

This leads to the difference equation in the time domain:

$$x(n) = \sum_{k=1}^{p} a_k x(n-k) + e(n)$$

This is excatly the same as a one step prediction using Wiener filters from lecture 10.

Only difference is that we negate right hand side of the Wiener-Hopf equations to comply with the sign of FIR filter coeffecients.

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import acf
from scipy.signal import lfilter

def lpc(x, k):

    acf_values = acf(x, nlags=k+1)
    
    cov = np.zeros((k,k))
    cross = np.zeros(k)
    for i in range(k):
        # We negate the right hand side
        cross[i] = -acf_values[i+1]
        for j in range(k):
            cov[i, j] = acf_values[np.abs(i-j)]

    h = np.linalg.inv(cov) @ cross

    # The filter coefficients are negated(Again) to comply with Python implementation of convolution
    b = np.concatenate([[0], -h])   # delay + predictor
    a = [1]

    x_hat = lfilter(b, a, x)

    e = x - x_hat
    return e, h, x_hat

e, h, x_hat  = lpc(x, k=15)

plot_num = 100

plt.figure()
plt.plot(x[:plot_num], label="x")
plt.plot(x_hat[:plot_num], label="x estimated")
plt.plot(e[:plot_num], label="error")
plt.legend()

plt.figure()
plt.plot(a, label="a")
plt.plot(h, label="a estimated")
plt.legend()
plt.title("Filter coeffecients")

    